In [ ]:
import torch
trajectory = torch.load("./hidden_states_last.pt",map_location=torch.device('cpu'))[0]

trajectory.shape # gives you torch.Size([4095, 1536])



torch.Size([4095, 1536])

In [ ]:


"""
gives you
Overall reconstruction accuracy:
using
    latent_dim = 64


or
Overall reconstruction accuracy:
using
    latent_dim = 32

"""

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(dim, dim),
            nn.LayerNorm(dim),
            nn.GELU(),
            # nn.Linear(dim, dim),
            # nn.Dropout(0.05),
            # nn.LayerNorm(dim)
        )

    def forward(self, x):
        return x + self.fc(x)

class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        dims = [input_dim]
        while dims[-1] // 2 >= latent_dim:
            dims.append(dims[-1] // 2)
        dims.append(latent_dim)

        layers = []
        for in_dim, out_dim in zip(dims[:-1], dims[1:]):
            layers.append(nn.Linear(in_dim, out_dim))
            if out_dim != latent_dim:
                layers.append(nn.LayerNorm(out_dim))
                layers.append(nn.GELU())
                layers.append(ResidualBlock(out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class Decoder(nn.Module):
    def __init__(self, output_dim, latent_dim):
        super().__init__()
        dims = [latent_dim]
        while dims[-1] * 2 <= output_dim:
            dims.append(dims[-1] * 2)
        dims.append(output_dim)

        layers = []
        for in_dim, out_dim in zip(dims[:-1], dims[1:]):
            layers.append(nn.Linear(in_dim, out_dim))
            if out_dim != output_dim:
                layers.append(nn.LayerNorm(out_dim))
                layers.append(nn.GELU())
                layers.append(ResidualBlock(out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

def train_autoencoder(trajectory, latent_dim=64, batch_size=128, num_epochs=100, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # trajectory_lengths = [t.shape[0] for t in trajectory]
    input_dim = trajectory.shape[1]

    # stacked_trajectory = torch.cat(trajectory, dim=0)

    # Create dataset and loader
    dataset = TensorDataset(trajectory)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize models
    encoder = Encoder(input_dim, latent_dim).to(device)
    decoder = Decoder(input_dim, latent_dim).to(device)

    optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=lr)
    criterion = nn.MSELoss()
    best_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        encoder.train()
        decoder.train()
        epoch_loss = 0.0

        for (batch,) in loader:
            x = batch.to(device)

            # Forward pass
            z = encoder(x)
            x_recon = decoder(z)

            # Compute loss
            loss = criterion(x_recon, x)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * x.size(0)

        epoch_loss /= len(dataset)

        # Evaluation
        if epoch % 10 == 0 or epoch == num_epochs:
            encoder.eval()
            decoder.eval()

            with torch.no_grad():
                # Compute Euclidean distance
                val_batch = trajectory.to(device)
                val_recon = decoder(encoder(val_batch))
                # l2_dist = torch.norm(val_recon - val_batch, dim=1).mean().item()

                # Cosine similarity
                cos_sim = nn.functional.cosine_similarity(val_recon, val_batch, dim=1).mean().item()

                print(f"Epoch {epoch:03d} | Loss: {epoch_loss:.6f} | Cosine Sim: {cos_sim:.6f}")

            if epoch_loss < best_loss:
                best_loss = epoch_loss

    return encoder, decoder

def encode_trajectory(encoder, trajectory, device=None):
    """
    Encode the entire trajectory tensor at once.
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    encoder.eval()
    with torch.no_grad():
        encoded_trajectory = encoder(trajectory.to(device))

    return encoded_trajectory

def decode_trajectory(decoder, encoded_trajectory, device=None):
    """
    Decode the entire encoded trajectory tensor at once. Ensures input is at least 2D.
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    decoder.eval()
    with torch.no_grad():
        # Ensure input is at least 2D (batch, features)
        if encoded_trajectory.dim() == 1:
            encoded_trajectory = encoded_trajectory.unsqueeze(0)
        decoded_trajectory = decoder(encoded_trajectory.to(device))

    return decoded_trajectory

def evaluate_reconstruction(original_trajectory, decoded_trajectory):
    """
    Evaluate reconstruction quality for the entire trajectory tensor.
    """
    if original_trajectory.device != decoded_trajectory.device:
        decoded_trajectory = decoded_trajectory.to(original_trajectory.device)

    # If input is 1D, unsqueeze to 2D for consistent metric calculation
    if original_trajectory.dim() == 1:
        original_trajectory = original_trajectory.unsqueeze(0)
    if decoded_trajectory.dim() == 1:
        decoded_trajectory = decoded_trajectory.unsqueeze(0)

    # # L2 distance
    # l2_dist = torch.norm(decoded_trajectory - original_trajectory, dim=1).mean().item()

    # Cosine similarity
    cos_sim = nn.functional.cosine_similarity(decoded_trajectory, original_trajectory, dim=1).mean().item()

    return {
        # "avg_l2_distance": l2_dist,
        "avg_cosine_similarity": cos_sim
    }


def calculate_reconstruction_accuracy(encoder, decoder, trajectory, device=None):
    """
    Calculate reconstruction accuracy as a percentage for the PyTorch autoencoder
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Get encoded and decoded trajectory
    encoded_trajectory = encode_trajectory(encoder, trajectory, device)
    decoded_trajectory = decode_trajectory(decoder, encoded_trajectory, device)

    original_data = trajectory.cpu().numpy()
    reconstructed_data = decoded_trajectory.cpu().numpy()

    mse = np.mean(np.square(original_data - reconstructed_data))
    variance = np.var(original_data)
    accuracy_percentage = (1 - mse/variance) * 100

    # Calculate component-wise accuracy
    component_mse = np.mean(np.square(original_data - reconstructed_data), axis=0)
    component_var = np.var(original_data, axis=0)
    component_accuracy = np.mean((1 - component_mse/component_var) * 100)

    metrics = evaluate_reconstruction(trajectory, decoded_trajectory)

    print("\nReconstruction Accuracy Metrics:")
    print(f"Overall reconstruction accuracy: {accuracy_percentage:.2f}%")
    print(f"Average component-wise accuracy: {component_accuracy:.2f}%")
    # print(f"Average L2 Distance: {metrics['avg_l2_distance']:.6f}")
    print(f"Average Cosine Similarity: {metrics['avg_cosine_similarity']:.6f}")

    return {
        "overall_accuracy": accuracy_percentage,
        "component_accuracy": (1 - component_mse/component_var) * 100,
        # "l2_distance": metrics["avg_l2_distance"],
        "cosine_similarity": metrics["avg_cosine_similarity"],
        "mse": mse
    }



In [ ]:
trajectory[0].shape

torch.Size([1536])

In [ ]:
if __name__ == "__main__":
    # Hyperparameters
    latent_dim = 128  # Target reduced dimension
    batch_size = 64
    lr = 1e-3
    num_epochs = 100

    encoder, decoder = train_autoencoder(
        trajectory=trajectory,
        latent_dim=latent_dim,
        batch_size=batch_size,
        num_epochs=num_epochs,
        lr=lr
    )

    accuracy_metrics = calculate_reconstruction_accuracy(encoder, decoder, trajectory)



Epoch 010 | Loss: 2.164499 | L2 Dist: 54.098827 | Cosine Sim: 0.826256
Epoch 020 | Loss: 1.472905 | L2 Dist: 43.233139 | Cosine Sim: 0.886581
Epoch 030 | Loss: 1.151322 | L2 Dist: 37.557602 | Cosine Sim: 0.914403
Epoch 040 | Loss: 0.931924 | L2 Dist: 33.475101 | Cosine Sim: 0.932438
Epoch 050 | Loss: 0.775821 | L2 Dist: 30.296339 | Cosine Sim: 0.945106
Epoch 060 | Loss: 0.672278 | L2 Dist: 28.389076 | Cosine Sim: 0.952792
Epoch 070 | Loss: 0.592351 | L2 Dist: 26.875671 | Cosine Sim: 0.958523
Epoch 080 | Loss: 0.512141 | L2 Dist: 24.584215 | Cosine Sim: 0.965322
Epoch 090 | Loss: 0.455247 | L2 Dist: 23.378595 | Cosine Sim: 0.968979
Epoch 100 | Loss: 0.415229 | L2 Dist: 22.150261 | Cosine Sim: 0.972448

Reconstruction Accuracy Metrics:
Overall reconstruction accuracy: 94.54%
Average component-wise accuracy: 85.81%
Average L2 Distance: 22.150261
Average Cosine Similarity: 0.972448


In [ ]:
if __name__ == "__main__":
    # Hyperparameters
    latent_dim = 64  # Target reduced dimension
    batch_size = 64
    lr = 1e-3
    num_epochs = 300

    encoder, decoder = train_autoencoder(
        trajectory=trajectory,
        latent_dim=latent_dim,
        batch_size=batch_size,
        num_epochs=num_epochs,
        lr=lr
    )

    accuracy_metrics = calculate_reconstruction_accuracy(encoder, decoder, trajectory)



Epoch 010 | Loss: 3.583819 | L2 Dist: 71.893166 | Cosine Sim: 0.695943
Epoch 020 | Loss: 2.709053 | L2 Dist: 61.984936 | Cosine Sim: 0.776878
Epoch 030 | Loss: 2.299956 | L2 Dist: 56.233173 | Cosine Sim: 0.812155
Epoch 040 | Loss: 2.035281 | L2 Dist: 52.203121 | Cosine Sim: 0.836630
Epoch 050 | Loss: 1.852918 | L2 Dist: 50.156353 | Cosine Sim: 0.848085
Epoch 060 | Loss: 1.722555 | L2 Dist: 47.375313 | Cosine Sim: 0.863865
Epoch 070 | Loss: 1.725615 | L2 Dist: 47.210846 | Cosine Sim: 0.865913
Epoch 080 | Loss: 1.553553 | L2 Dist: 44.222549 | Cosine Sim: 0.879287
Epoch 090 | Loss: 1.448157 | L2 Dist: 42.270977 | Cosine Sim: 0.889200
Epoch 100 | Loss: 1.428922 | L2 Dist: 42.197643 | Cosine Sim: 0.890330
Epoch 110 | Loss: 1.311438 | L2 Dist: 39.971607 | Cosine Sim: 0.900601
Epoch 120 | Loss: 1.289553 | L2 Dist: 39.447876 | Cosine Sim: 0.904458
Epoch 130 | Loss: 1.216488 | L2 Dist: 38.596214 | Cosine Sim: 0.908966
Epoch 140 | Loss: 1.178372 | L2 Dist: 37.997829 | Cosine Sim: 0.911829
Epoch 

In [ ]:
if __name__ == "__main__":
    # Hyperparameters
    latent_dim = 32  # Target reduced dimension
    batch_size = 64
    lr = 1e-3
    num_epochs = 300

    encoder, decoder = train_autoencoder(
        trajectory=trajectory,
        latent_dim=latent_dim,
        batch_size=batch_size,
        num_epochs=num_epochs,
        lr=lr
    )

    accuracy_metrics = calculate_reconstruction_accuracy(encoder, decoder, trajectory)



Epoch 010 | Loss: 3.114817 | L2 Dist: 66.351280 | Cosine Sim: 0.740156
Epoch 020 | Loss: 2.861269 | L2 Dist: 64.184189 | Cosine Sim: 0.758287
Epoch 030 | Loss: 2.839603 | L2 Dist: 63.470081 | Cosine Sim: 0.763982
Epoch 040 | Loss: 2.831652 | L2 Dist: 62.443546 | Cosine Sim: 0.770601
Epoch 050 | Loss: 2.657126 | L2 Dist: 61.149544 | Cosine Sim: 0.780942
Epoch 060 | Loss: 2.756527 | L2 Dist: 61.567329 | Cosine Sim: 0.774983
Epoch 070 | Loss: 2.668292 | L2 Dist: 60.274960 | Cosine Sim: 0.784556
Epoch 080 | Loss: 2.540964 | L2 Dist: 59.017342 | Cosine Sim: 0.791408
Epoch 090 | Loss: 2.789755 | L2 Dist: 63.970779 | Cosine Sim: 0.760134
Epoch 100 | Loss: 2.791267 | L2 Dist: 63.636669 | Cosine Sim: 0.758669
Epoch 110 | Loss: 2.610243 | L2 Dist: 60.317879 | Cosine Sim: 0.783802
Epoch 120 | Loss: 2.760078 | L2 Dist: 62.243565 | Cosine Sim: 0.769706
Epoch 130 | Loss: 2.672255 | L2 Dist: 60.666290 | Cosine Sim: 0.778828
Epoch 140 | Loss: 2.519322 | L2 Dist: 58.571941 | Cosine Sim: 0.794818
Epoch 

In [ ]:
if __name__ == "__main__":
    # Hyperparameters
    latent_dim = 32  # Target reduced dimension
    batch_size = 64
    lr = 3e-4
    num_epochs = 300

    encoder, decoder = train_autoencoder(
        trajectory=trajectory,
        latent_dim=latent_dim,
        batch_size=batch_size,
        num_epochs=num_epochs,
        lr=lr
    )

    accuracy_metrics = calculate_reconstruction_accuracy(encoder, decoder, trajectory)



Epoch 010 | Loss: 1.600662 | L2 Dist: 45.431129 | Cosine Sim: 0.876870
Epoch 020 | Loss: 1.167119 | L2 Dist: 37.785721 | Cosine Sim: 0.913027
Epoch 030 | Loss: 0.923650 | L2 Dist: 33.536152 | Cosine Sim: 0.932007
Epoch 040 | Loss: 0.773998 | L2 Dist: 30.412956 | Cosine Sim: 0.944821
Epoch 050 | Loss: 0.661172 | L2 Dist: 28.461849 | Cosine Sim: 0.952733
Epoch 060 | Loss: 0.590631 | L2 Dist: 26.495779 | Cosine Sim: 0.959184
Epoch 070 | Loss: 0.503051 | L2 Dist: 24.469797 | Cosine Sim: 0.965523
Epoch 080 | Loss: 0.449671 | L2 Dist: 23.674177 | Cosine Sim: 0.968087
Epoch 090 | Loss: 0.387402 | L2 Dist: 21.979719 | Cosine Sim: 0.972700
Epoch 100 | Loss: 0.375536 | L2 Dist: 21.507402 | Cosine Sim: 0.974143
Epoch 110 | Loss: 0.342940 | L2 Dist: 21.231283 | Cosine Sim: 0.975092
Epoch 120 | Loss: 0.296930 | L2 Dist: 19.340391 | Cosine Sim: 0.979411
Epoch 130 | Loss: 0.299347 | L2 Dist: 19.156448 | Cosine Sim: 0.979858
Epoch 140 | Loss: 0.280858 | L2 Dist: 18.488655 | Cosine Sim: 0.981384
Epoch 

In [ ]:
if __name__ == "__main__":
    # Hyperparameters
    latent_dim = 16  # Target reduced dimension
    batch_size = 64
    lr = 3e-4
    num_epochs = 400

    encoder, decoder = train_autoencoder(
        trajectory=trajectory,
        latent_dim=latent_dim,
        batch_size=batch_size,
        num_epochs=num_epochs,
        lr=lr
    )

    accuracy_metrics = calculate_reconstruction_accuracy(encoder, decoder, trajectory)



Epoch 010 | Loss: 1.979391 | L2 Dist: 52.172646 | Cosine Sim: 0.839586
Epoch 020 | Loss: 1.606312 | L2 Dist: 45.221500 | Cosine Sim: 0.874979
Epoch 030 | Loss: 1.501450 | L2 Dist: 44.264866 | Cosine Sim: 0.880434
Epoch 040 | Loss: 1.251300 | L2 Dist: 39.306259 | Cosine Sim: 0.903550
Epoch 050 | Loss: 1.165139 | L2 Dist: 37.651520 | Cosine Sim: 0.911714
Epoch 060 | Loss: 1.093688 | L2 Dist: 36.474380 | Cosine Sim: 0.916396
Epoch 070 | Loss: 1.002714 | L2 Dist: 34.876366 | Cosine Sim: 0.923919
Epoch 080 | Loss: 1.083221 | L2 Dist: 37.503555 | Cosine Sim: 0.914468
Epoch 090 | Loss: 0.904012 | L2 Dist: 32.685047 | Cosine Sim: 0.934361
Epoch 100 | Loss: 0.872475 | L2 Dist: 32.080933 | Cosine Sim: 0.937425
Epoch 110 | Loss: 0.809595 | L2 Dist: 31.378166 | Cosine Sim: 0.941163
Epoch 120 | Loss: 0.816686 | L2 Dist: 30.341703 | Cosine Sim: 0.944353
Epoch 130 | Loss: 0.693855 | L2 Dist: 28.946356 | Cosine Sim: 0.950046
Epoch 140 | Loss: 0.647619 | L2 Dist: 28.518448 | Cosine Sim: 0.951559
Epoch 

lr is very important and makes a huge difference.